In [1]:
import subprocess
import sys


def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try: 
    import requests
except: 
    install('requests')
    import requests
    
try: 
    import selenium
except: 
    install('selenium')
    import selenium

try: 
    from bs4 import BeautifulSoup
except: 
    install('beautifulsoup4')
    from bs4 import BeautifulSoup
    
try: 
    from webdriver_manager.firefox import GeckoDriverManager
except:
    install('webdriver-manager')
    from webdriver_manager.firefox import GeckoDriverManager
    

path = GeckoDriverManager().install()


import json
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options as Options
from threading import Thread
import time
import tkinter as tk
from tkinter.tix import *
from tkinter import *
from tkinter import ttk
from tkinter import messagebox



====== WebDriver manager ======
Current firefox version is 95.0
Get LATEST geckodriver version for 95.0 firefox
Driver [/home/dd/.wdm/drivers/geckodriver/linux64/v0.30.0/geckodriver] found in cache


In [2]:
opts = Options()
opts.add_argument("--headless")
opts.add_argument('--no-sandbox')
opts.add_argument('--no-default-browser-check')
opts.add_argument('--disable-gpu')
opts.add_argument('--disable-extensions')
opts.add_argument('--disable-default-apps')

driversA = []
driversB = []
free_trademark= []
free_local_number = []
free_toll_free = []
free_domain = []
limit = 3

In [3]:
def runDriver():
    return Firefox(executable_path=path, options=opts)   

In [4]:
def initDriver():
    global driversA, driversB, initDone 
    driversA.append(runDriver())
    driversB.append(runDriver())
    
    initDone +=1

In [5]:
def init(frame):
    global free, free_numbers, drivers, initDone
    initDone = 0
    
    for i in range(limit):
            Thread(target = initDriver).start()
            free_trademark.append(True)
            free_local_number.append(True)
            free_toll_free.append(True)
            free_domain.append(True)
    
    while initDone < limit:
#         time.sleep(3)
        continue
        
    frame.destroy()
    lookupScreen()

In [6]:
def domain(keyword):
    base = f"https://api.godaddy.com/v1/domains/available?domain={keyword}.com&checkType=FAST&forTransfer=false"
    api_key = "sso-key fYWJ9TjpSa5N_AQWH9qps2mtJWBoyFyvMWE:RjtwUn7mJp3wroKtGW597Z"

    resp = requests.get(base, 
                        headers={'accept':'application/json',
                   'Authorization': api_key})
    resp = json.loads(resp.text)
    
    return resp['available']

In [7]:
def tollfree(free, t, kw, frame, pos):
    global done
    
    if len(kw) < 7:
            Notify("TOLL-FREE NUMBER", frame, pos, 6, color ="black")
            done += 1
            free[t] = True
            return
        
    base = f'https://www.ringboost.com/toll-free/vanity/search-results?phone={kw}' 
    resp = requests.get(base)
    soup = BeautifulSoup(resp.text)

    results = soup.find_all('div', {"class":"number"})
    kt = False
    for each in results:
        if kw.replace(' ','') in each.text.replace('-', ''):
            kt = True
            break
    
    if kt: Notify("TOLL-FREE NUMBER", frame, pos, 6, color ="green")
    else: Notify("TOLL-FREE NUMBER", frame, pos, 6, color ="red")
        
    done += 1
    free[t] = True

In [8]:
def localnumber(driver, free, t, kw, frame, pos):
    global done
    
    if len(kw) < 7:
            Notify("LOCAL NUMBER", frame, pos, 5, color="black")
            done += 1
            free[t] = True
            return 
        
    base = f"https://www.numberbarn.com/search?search={kw}"
    driver.get(base)
    time.sleep(5)
    resp = driver.find_element_by_xpath('//span[@class="count"]')
    
    if len(resp.text) > 0:
        Notify("LOCAL NUMBER", frame, pos, 5, color="green")
    else: 
        Notify("LOCAL NUMBER", frame, pos, 5, color="red")
        
    done += 1
    free[t] = True

In [9]:
def UpstoValidate(drivers, free, index, keyword, frame, pos):
    base = 'https://tmsearch.uspto.gov/bin/gate.exe?f=login&p_lang=english&p_d=trmk' 
    driver = drivers[index]
    
    try:
        driver.get(base)
        driver.find_element_by_css_selector('body > center > table:nth-child(1) > tbody > tr:nth-child(2) > td > font > font > a').click()
        driver.find_element_by_css_selector('#querytext > input[type=text]').send_keys(keyword)
        driver.find_element_by_css_selector('body > form > font > table:nth-child(6) > tbody > tr:nth-child(5) > td > input[type=SUBMIT]:nth-child(4)').click()
    
        if "Error" in driver.title: 
            Notify("TRADEMARK", frame, pos, 4, color='GREEN')
        else: 
            Notify("TRADEMARK", frame, pos, 4, color='RED')

        global done
        done+=1
        free[index] = True
    except:
        driver.close()
        drivers[index] = runDriver()
        time.sleep(30)
        UpstoValidate(drivers, free, index, keyword, frame, pos)

In [10]:
def DomainValidate(free, t, keyword, frame, pos):
    if domain(keyword):
        Notify("DOMAIN", frame, pos, 3, color="green")
    else: 
        Notify("DOMAIN", frame, pos, 3, color="red")
        
    global done
    done+=1
    free[t] = True

In [11]:
def LookUp(keywords, numbers, domains, frame, button):
    global done
    done = 0
    visited_trademark = 0
    visited_local_number = 0
    visited_toll_free = 0
    visited_domain = 0
    
    while visited_trademark + visited_local_number + visited_toll_free + visited_domain \
    < len(keywords) + len(numbers)*2 + len(domains):
        
        if visited_trademark < len(keywords):
            for t in range(limit):
                if free_trademark[t] and visited_trademark < len(keywords):
                    free_trademark[t] = False
                    Thread(target=UpstoValidate, args=(driversA, free_trademark, t, \
                                                       keywords[visited_trademark], frame, visited_trademark)).start()
                    visited_trademark+=1
        
        if visited_local_number < len(numbers):
            for t in range(limit):
                if free_local_number[t] and visited_local_number < len(numbers):
                    free_local_number[t] = False
                    Thread(target=localnumber, args=(driversB[t], free_local_number, t, \
                                                     numbers[visited_local_number], frame, visited_local_number)).start()
                    visited_local_number+=1
        
        if visited_toll_free < len(numbers):
            for t in range(limit):
                if free_toll_free[t] and visited_toll_free < len(numbers):
                    free_toll_free[t] = False
                    Thread(target=tollfree, args=(free_toll_free, t, \
                                                     numbers[visited_toll_free], frame, visited_toll_free)).start()
                    visited_toll_free+=1
        
        if visited_domain < len(domains):
            for t in range(limit):
                if free_domain[t] and visited_domain < len(domains):
                    free_domain[t] = False
                    Thread(target=DomainValidate, args=(free_domain, t, \
                                                        domains[visited_domain], frame, visited_domain)).start()
                    visited_domain+=1
    
    while done < len(keywords) + len(numbers)*2 + len(domains): 
        time.sleep(2*len(keywords)/limit)
        continue
        
    button['state'] = NORMAL
    button['text'] = "Look up"

In [12]:
def Notify(msg, frame, row, col, color = None):
    label = Message(master = frame, text = msg, bg = color, width = 900, justify = CENTER, font = ("Arial", 14))
    label.grid(row=row, column=col, sticky=W, pady = 3, padx = 2)

In [13]:
def initScreen():    
    frame = Frame()
    frame.pack()  
    label = Message(master = frame, 
                     text="This is a mandatory step, currently the system only supports GeckoDriver, please install Firefox browser if you don't have!",
                    font = ("Arial", 17), width = 800, justify = CENTER)
    label.pack()
        
    def ok():
        label = Message(master = frame, 
                         text="Initializing...",
                        font = ("Arial", 17), width = 800, justify = CENTER)
        label.pack()
                
        Thread(target = init, args = (frame, )).start()
        
            
    button = Button(frame, text="Click here to initialize system!", font = ("Arial", 12, "bold"), command=ok)
    button.pack(pady = 15)


In [14]:
def lookupScreen():
    def myfunction(event):
        canvas.configure(scrollregion=canvas.bbox("all"),width=200,height=200)

    frame1 = Frame()
    frame1.pack()
    frame2 = Frame()
    frame2.pack()
    frame3 = Frame()
    frame3.pack(fill = BOTH, expand = True)
    
    canvas=Canvas(frame3, highlightthickness=0)
    window=Frame(canvas)
    myscrollbar=Scrollbar(frame3,orient="vertical",command=canvas.yview, width = 10)
    canvas.configure(yscrollcommand=myscrollbar.set)

    myscrollbar.pack(side="right",fill="y")
    canvas.pack(side="left", fill=tk.BOTH, expand=True)
    canvas.create_window((0,0),window=window, anchor='nw')
    window.bind("<Configure>",myfunction)
    
    variable = StringVar()
    font = ("Arial", 13)
    
    w = Entry(master = frame1, textvariable = variable, font = font, width = 70)
    w.pack(side = LEFT)
    
    
    def toNum(kw):
        kw = kw.upper()
        kw = list(kw)
        for i in range(len(kw)):
            if kw[i] in 'ABC':
                kw[i] = '2'
            elif kw[i] in 'DEF':
                kw[i] = '3'
            elif kw[i] in 'GHI':
                kw[i] = '4'
            elif kw[i] in 'JKL':
                kw[i] = '5'
            elif kw[i] in 'MNO':
                kw[i] = '6'
            elif kw[i] in 'PQRS':
                kw[i] = '7'
            elif kw[i] in 'TUV':
                kw[i] = '8'
            elif kw[i] in 'WXYZ':
                kw[i] = '9'
            elif kw[i] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']:
                continue
            else: kw[i] = ''
        
        kw = ''.join(kw)
        if len(kw) > 7:
            kw = kw[:7]
        
        return kw
        
    def ok(frame):
        for each in frame.winfo_children():
            each.destroy()
        kws = variable.get().split(',')
        numbers = []
        keywords = []
        domains = []
        
        skipped = 0
        for i in range(len(kws)):
            kw = kws[i].strip().lstrip().rstrip()
            if kw != '':
                num = toNum(kw)
                kw = ''.join(ch for ch in kw if ch.isalnum() or ch == ' ')
                dom = ''.join(ch for ch in kw if ch.isalnum())
                
                Notify(kw, frame, i - skipped, 0)
                Notify(dom.lower() + '.com', frame, i-skipped, 1)
                Notify(num, frame, i - skipped, 2)
                
                keywords.append(kw)
                numbers.append(num)
                domains.append(dom.lower())
            else: 
                skipped+=1
        button['text'] = "Searching"
        button['state'] = DISABLED
        
        Thread(target = LookUp, args = (keywords, numbers, domains, frame, button,)).start()
        
    button = Button(frame1, text="Look up", font = ("Arial", 12, "bold"), command=lambda: ok(window))
    button.pack(padx = 10, side = LEFT)
    
    
    
    label = Message(master = frame2, 
                     text="Hint: If you search too many words in a short period of time, or the searched words are repeated many times, there is a possibility of being temporarily blocked from UPSTO. But please be patient, the system will wait and continue searching when possible.",
                    font = ("Arial", 10, "italic"), width = 900, justify = CENTER)
    label.pack(pady = 5, side = TOP)
    

In [21]:
root = Tk()
root.title('InfoMagic - Keywords Look Up v1.0')
root.geometry('900x600')

try:
    # root.resizable(0, 0)

    initScreen()
    # lookupScreen()

    def on_closing():
        if messagebox.askokcancel("Quit", "Do you want to quit?"):
            for each in driversA:
                try:
                    each.close()
                except: 
                    pass
            for each in driversB:
                try: 
                    each.close()
                except: 
                    pass

            root.destroy()
            
    root.protocol("WM_DELETE_WINDOW", on_closing)
    root.mainloop()
except:
    for each in driversA:
        try:
            each.close()
        except: 
            pass
    for each in driversB:
        try: 
            each.close()
        except: 
            pass
    root.destroy()